# Colab Imports

In [78]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Installation

In [79]:
!pip -q install langgraph langchain_openai langchain_community tiktoken konlpy langchain-pinecone pinecone-notebooks pyppeteer

# Settings & Loadings

In [80]:
import os
import json
import pandas as pd

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

PROFECT_DIR = os.path.join("/content/drive/MyDrive", "Profect SAE")
CSVs = os.path.join(PROFECT_DIR, "test_data/CSVs")
files = os.listdir(CSVs)
csv_files = [
    file for file in files
      if file.endswith(".csv")
]

df_one = pd.read_csv(os.path.join(CSVs, csv_files[0]))
print(df_one.iloc[0])

title      [종합] 김태희, 해외서 권총 강도 당했다…'연매출 1100억' 정샘물 "피해액 2...
url        https://m.entertain.naver.com/article/312/0000...
content    본문 바로가기 naver 엔터 뉴스 스포츠 사용자 링크 로그인 검색 홈 드라마 영화...
Name: 0, dtype: object


# Things I need to do to add feedback loop

1. New(or refactored) state and state management logic
2. Applying Short-term memory for feedback: CheckPointer
  - [LangGraph Memory](https://langchain-ai.github.io/langgraph/concepts/memory/)
  - [LangGraph CheckPointer with InMemorySaver](https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpoints)
3. Modifying Generation Flows based on 5W1H below.
  - Who?(Target): User
  - When?: After stocks are generated and returned.
  - Where?: (Feedback) Chrome Extension
  - What?: Whether user liked it or not.
  - How?: By adding a field at the state and use `InMemoryStore or Saver`
    - `InMemorySaver`: Per Thread aka per agent -> that is why it's part of checkpointer~ yay
    - `InMemoryStore`: Cross Generation aka throught out the generation
  - Why?: To improve the result.

4. Step By Step
  - new state
  - new Feedback Agent or modified RAG Agent
  - (2025.04.28) Notes
    - Preprocessing -> pass by dict
    - RAG -> Save results, preprocessed, original? `InMemoryStore or Saver`
      - `InMemoryStore`: Need results through out the generation and needs to be used later(in the future)
    - Feedback
      - score -> for usage data
      - regeneration

## Schema, DTO & State

### Schema
- `Stock`

In [81]:
from pydantic import BaseModel, Field
from typing import Optional, List

class Stock(BaseModel):
  korean_name: str
  english_name: str
  market: str
  ticker_code: str

### DTO
- `StockGenRequestDto`
- `FeedbackRequestDto`
- `StockGenResponseDto`

In [82]:
from pydantic import BaseModel, Field
from typing import Optional, List

class StockGenRequestDto(BaseModel):
  original_title: str = Field(..., min_length=1)
  original_url: str = Field(..., min_length=5, max_length=2083)
  original_content: str = Field(..., min_length=1)

class FeedbackRequestDto(BaseModel):
  state_id: str = Field(..., min_length=1)

class StockGenResponseDto(BaseModel):
  state_id: str = Field(..., min_length=1)
  generated_stocks: list[Stock]

### States
- `PreprocessingFlowState`
- `RAGFlowState`
- `FeedbackFlowState`
- `FlowState`: Pending

In [83]:
from pydantic import BaseModel, Field
from typing import Optional, List

class PreprocessingFlowState(BaseModel):
  state_id: str = Field(..., min_length=1)
  request_dto: StockGenRequestDto
  route: str
  preprocessed_title: str
  preprocessed_url: str
  preprocessed_content: str

class RAGFlowState(BaseModel):
  state_id: str = Field(..., min_length=1)
  preprocessed_result: dict
  response_dto: list[Stock]


class FeedbackFlowState(BaseModel):
  state_id: str = Field(..., min_length=1)
  feedback_score: int
  regeneration: bool
  preprocessed_result: dict
  rag_result: dict

## Nodes

### Preprocessing

#### Tools
- `text_cleaning`

In [84]:
def text_cleaning(text: str) -> str:
  """Cleaning text by removing unnecessary characters and spaces."""
  if isinstance(text, str):
      # Convert to lowercase
      text = text.lower()
      # Remove leading and trailing spaces
      text = text.strip()
      # Replace multiple spaces with a single space
      text = re.sub(r'\s+', ' ', text)
      # Remove special characters except for common ones
      text = re.sub(r'[^\w\s.,!?$%&@()-]', '', text)
  return text

#### Router Node
- `prerocessing_router_node`

In [85]:
import re

def preprocessing_router_node(preprocessing_flow_state: PreprocessingFlowState):
  print("Routing")
  request_dto = preprocessing_flow_state.request_dto

  youtube_pattern = r"^https?://(www\.)?youtube\.com"
  naver_news_pattern = r"^https?://n\.news\.naver\.com"
  naver_entertainment_pattern = r"^https?://m\.entertain\.naver\.com"
  naver_sports_pattern = r"^https?://m\.sports\.naver\.com"

  if re.match(naver_news_pattern, request_dto.original_url):
      route = "naver_news"

  elif re.match(naver_entertainment_pattern, request_dto.original_url):
      route = "naver_entertainment"

  elif re.match(naver_sports_pattern, request_dto.original_url):
      route = "naver_sports"

  elif re.match(youtube_pattern, request_dto.original_url):
      route = "youtube"
  else:
      route = "default"

  # Basic text cleaning
  preprocessing_flow_state.preprocessed_title = text_cleaning(request_dto.original_title)
  preprocessing_flow_state.preprocessed_content = text_cleaning(request_dto.original_content)
  preprocessing_flow_state.preprocessed_url = request_dto.original_url
  preprocessing_flow_state.route = route
  return preprocessing_flow_state

#### Preprocessing Nodes Based on `flow_state.preprocessing_flow_state.route`
1. `naver_news_preprocessing_node`
2. `naver_entertainment_preprocessing_node`
3. `naver_sports_preprocessing_node`
4. `youtube_preprocessing_node`
5. `default_preprocessing_node`

---

1. `content_parser`

In [86]:
import re

def content_parser(text: str, remove_after: str = None, patterns: list = None) -> str:
  if not isinstance(text, str):
      return text

  if remove_after:
      text = re.sub(fr"{remove_after}.*", "", text, flags=re.IGNORECASE)

  if patterns:
    combined_pattern = "|".join(patterns)
    text = re.sub(combined_pattern, "", text, flags=re.IGNORECASE)

  # Remove excessive whitespace
  text = re.sub(r"\s+", " ", text).strip()

  return text

In [87]:
def preprocessing_naver_news_node(preprocessing_flow_state: PreprocessingFlowState):
  print("NEWS")
  requestDto = preprocessing_flow_state.request_dto
  """Cleans Naver News content."""
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, remove_after="copyright .*?\. all rights reserved")
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, remove_after="기사 공유하기")
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, patterns = [
          r"본문 바로가기", r"naver", r"뉴스", r"엔터", r"스포츠", r"날씨", r"프리미엄",
          r"사용자 링크", r"로그인", r"서비스", r"더보기", r"검색", r"언론사별", r"정치",
          r"경제", r"사회", r"생활문화", r"it과학", r"세계", r"랭킹", r"신문보기",
          r"오피니언", r"tv", r"팩트체크", r"알고리즘 안내", r"정정보도 모음"])
  return preprocessing_flow_state

def preprocessing_naver_entertainment_node(preprocessing_flow_state: PreprocessingFlowState):
  print("ENT")
  requestDto = preprocessing_flow_state.request_dto

  """Cleans Naver Entertainment content."""
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, remove_after="copyright .*?\. all rights reserved")
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, remove_after="기사 공유하기")
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, patterns = [
      r"본문 바로가기", r"naver", r"엔터", r"뉴스", r"스포츠", r"사용자 링크", r"로그인",
      r"검색", r"홈", r"드라마", r"영화", r"뮤직", r"연애", r"포토", r"랭킹", r"최신뉴스", r"연재", r"종합"])
  return preprocessing_flow_state

def preprocessing_naver_sports_node(preprocessing_flow_state: PreprocessingFlowState):
  print("SPORTS")
  requestDto = preprocessing_flow_state.request_dto
  """Cleans Naver Sports content."""
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, remove_after="copyright .*?\. all rights reserved")
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, remove_after="기사 공유하기")
  return preprocessing_flow_state

def preprocessing_youtube_node(preprocessing_flow_state: PreprocessingFlowState):
  print("YOUTUBE")
  requestDto = preprocessing_flow_state.request_dto
  """Cleans YouTube transcript content."""
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, remove_after="더보기")
  preprocessing_flow_state.preprocessed_content = content_parser(requestDto.original_content, patterns = [r"kr", r"탐색", r"건너뛰기", r"만들기"])
  return preprocessing_flow_state

### RAG

#### Nodes

In [88]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from pydantic import BaseModel
from typing import List
from langchain_openai import OpenAIEmbeddings
import tiktoken

pc = Pinecone(api_key=userdata.get('PINECONE-API-KEY'))


def retriever_init(index_name: str, embeddings, namespace):
  index = pc.Index(index_name)
  vector_store = PineconeVectorStore(index=index, embedding=embeddings)
  return vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 2, "score_threshold": 0.65,"namespace": namespace},)

def retrieved_vector_parser(given_vectors, market_name="N/A"):
    return [
        Stock(
            korean_name=vector.metadata.get("korean_name", "N/A"),
            english_name=vector.metadata.get("english_name", "N/A"),
            ticker_code=vector.metadata.get("ticker_code", "N/A"),
            market=market_name
        )
        for vector in given_vectors
    ]

def retriever_node(rag_flow_state: RAGFlowState):
  text = f"{rag_flow_state.preprocessed_result['title']}  {rag_flow_state.preprocessed_result['content']}"

  # Vector Store init
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
  kor_retriever = retriever_init("sae-embedded-stocks-kor", embeddings, "kospi-900")
  foriegn_retriever = retriever_init("sae-embedded-stocks-foriegn", embeddings, "nasdaq-1000")

  # Invoke
  kor_result = retrieved_vector_parser(kor_retriever.invoke(text), "KOSPI")
  foriegn_result = retrieved_vector_parser(foriegn_retriever.invoke(text),"NASDAQ")
  rag_flow_state.response_dto.extend(kor_result)
  rag_flow_state.response_dto.extend(foriegn_result)
  return rag_flow_state

### Feedback

#### Nodes

In [89]:
def feedback_router_node(feedback_flow_state: FeedbackFlowState):
  if (feedback_flow_state.feedback_score <= 3):
    feedback_flow_state.regeneration = True
  return feedback_flow_state

def regenerate_node(feedback_flow_state: FeedbackFlowState):
  return feedback_flow_state

def feedback_save_node(feedback_flow_state: FeedbackFlowState):
  return feedback_flow_state

## Flow

### Preprocessing Flow

In [90]:
from langgraph.graph import StateGraph, START, END

preprocessing_graph = StateGraph(state_schema=PreprocessingFlowState)

# Registering Noes

## router
preprocessing_graph.add_node("preprocessing_router_node", preprocessing_router_node)

## preprocessing
preprocessing_graph.add_node("naver_news_preprocessing_node", preprocessing_naver_news_node)
preprocessing_graph.add_node("naver_entertainment_preprocessing_node", preprocessing_naver_entertainment_node)
preprocessing_graph.add_node("naver_sports_preprocessing_node", preprocessing_naver_sports_node)
preprocessing_graph.add_node("youtube_preprocessing_node", preprocessing_youtube_node)


# Registering Edges
# Conditional Edges
preprocessing_graph.add_edge(START, "preprocessing_router_node")

preprocessing_graph.add_conditional_edges(
    "preprocessing_router_node",
    lambda flow_state: flow_state.route,
    {
        "naver_news": "naver_news_preprocessing_node",
        "naver_entertainment": "naver_entertainment_preprocessing_node",
        "naver_sports": "naver_sports_preprocessing_node",
        "youtube": "youtube_preprocessing_node",
        "default": END,
    }
)

preprocessing_graph.add_edge("naver_news_preprocessing_node", END)
preprocessing_graph.add_edge("naver_entertainment_preprocessing_node", END)
preprocessing_graph.add_edge("naver_sports_preprocessing_node", END)
preprocessing_graph.add_edge("youtube_preprocessing_node", END)

# agent Init
preprocessing_agent = preprocessing_graph.compile()

### RAG Flow

In [91]:
from langgraph.graph import StateGraph, START, END

rag_graph = StateGraph(state_schema=RAGFlowState)

# Registring Nodes
rag_graph.add_node("retriever_node", retriever_node)

# Registering Edges
rag_graph.add_edge(START, "retriever_node")
rag_graph.add_edge("retriever_node", END)

rag_agent = rag_graph.compile()

### Feedback Flow

In [92]:
from langgraph.graph import StateGraph, START, END

feedback_graph = StateGraph(state_schema=FeedbackFlowState)

feedback_graph.add_node("feedback_router_node", feedback_router_node)
feedback_graph.add_node("regenerate_node", regenerate_node)
feedback_graph.add_node("feedback_save_node", feedback_save_node)

feedback_graph.add_edge(START, "feedback_router_node")
feedback_graph.add_conditional_edges(
    "feedback_router_node",
    lambda flow_state: flow_state.regeneration,
    {
        True: "regenerate_node",
        False: "feedback_save_node",
    }
)

feedback_graph.add_edge("regenerate_node", END)
feedback_graph.add_edge("feedback_save_node", END)

feedback_agent = feedback_graph.compile()

## Visualization
- prerocessing flow
- rag flow

In [93]:
# Visualize your graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
print(preprocessing_agent.get_graph().draw_mermaid())
# display(Image(preprocessing_agent.get_graph().draw_mermaid_png()))

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	preprocessing_router_node(preprocessing_router_node)
	naver_news_preprocessing_node(naver_news_preprocessing_node)
	naver_entertainment_preprocessing_node(naver_entertainment_preprocessing_node)
	naver_sports_preprocessing_node(naver_sports_preprocessing_node)
	youtube_preprocessing_node(youtube_preprocessing_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> preprocessing_router_node;
	preprocessing_router_node -. &nbsp;default&nbsp; .-> __end__;
	preprocessing_router_node -. &nbsp;naver_entertainment&nbsp; .-> naver_entertainment_preprocessing_node;
	preprocessing_router_node -. &nbsp;naver_news&nbsp; .-> naver_news_preprocessing_node;
	preprocessing_router_node -. &nbsp;naver_sports&nbsp; .-> naver_sports_preprocessing_node;
	preprocessing_router_node -. &nbsp;youtube&nbsp; .-> youtube_preprocessing_node;
	naver_entertainment_preprocessing_node --> __end__;
	naver_news_preprocessing_

In [94]:
# Visualize your graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

print(rag_agent.get_graph().draw_mermaid())
# display(Image(rag_agent.get_graph().draw_mermaid_png()))

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retriever_node(retriever_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> retriever_node;
	retriever_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [95]:
# Visualize your graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

print(feedback_agent.get_graph().draw_mermaid())
# display(Image(feedback_agent.get_graph().draw_mermaid_png()))

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	feedback_router_node(feedback_router_node)
	regenerate_node(regenerate_node)
	feedback_save_node(feedback_save_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> feedback_router_node;
	feedback_router_node -. &nbsp;False&nbsp; .-> feedback_save_node;
	feedback_router_node -. &nbsp;True&nbsp; .-> regenerate_node;
	feedback_save_node --> __end__;
	regenerate_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



## Tesing Agents

### Testing Preprocessing Agents

In [ ]:
import uuid
import time
from langchain.storage import InMemoryStore
import random

preprocessing_store = InMemoryStore()
rag_store = InMemoryStore()

count = 0

for row in df_one.itertuples(index=False):
  start = time.time()

  # Preprocessing Agent
  ## State Init
  stock_gen_request_dto = StockGenRequestDto(
      original_title=row.title,
      original_url=row.url,
      original_content=row.content
  )

  preprocessing_flow_state = PreprocessingFlowState(
      state_id=str(uuid.uuid1()),
      request_dto=stock_gen_request_dto,
      route="",
      preprocessed_title="",
      preprocessed_url="",
      preprocessed_content="")

  print("Title: ", preprocessing_flow_state.request_dto.original_title + "\n")
  print("Preprocessing Agent")
  preprocessed_result = preprocessing_agent.invoke(preprocessing_flow_state)
  before_length = len(preprocessed_result['request_dto'].original_content)
  after_length = len(preprocessed_result['preprocessed_content'])
  print("Before: "+str(before_length))
  print("After: " +str(after_length))
  print("Storing Preprocessing Agnent's Result: " + preprocessed_result['state_id'])
  preprocessing_store.mset(
      [(preprocessed_result['state_id'], preprocessed_result)]
  )
  print("")

  # RAG Agent
  print("RAG Agent")
  rag_flow_state = RAGFlowState(
      state_id=preprocessed_result['state_id'],
      preprocessed_result={
          "title": preprocessed_result['preprocessed_title'],
          "url": preprocessed_result['preprocessed_url'],
          "content": preprocessed_result['preprocessed_content']
      },
      response_dto=[]
  )
  rag_result = rag_agent.invoke(rag_flow_state)
  print("Storing RAG Agent's Result: " + rag_result['state_id'])
  rag_store.mset(
      [(rag_result['state_id'], rag_result)]
      )
  print("Retrieved stocks")
  for stock in rag_result['response_dto']:
    print(stock.korean_name, stock.english_name, stock.market, stock.ticker_code)
  elapsed = time.time() - start
  print("")
  print(f"Elapsed Time for Generation: {elapsed:.2f} seconds")

  # Feedback Agent
  if random.getrandbits(1) == 0:
    feedback_start = time.time()
    print("")
    print("Feedback Requested")
    feedback_score = random.randint(1, 5)
    ## Feedback State Init
    print("Given Feedback Score: " + str(feedback_score))
    print("State ID: " + rag_result['state_id'])
    requested_state_id = rag_result['state_id']
    preprocessing_flow_state = preprocessing_store.mget([requested_state_id])[0]
    rag_flow_state = rag_store.mget([requested_state_id])[0]
    print("Feedback Agent")
    feedback_flow_state = FeedbackFlowState(
        state_id=requested_state_id,
        feedback_score=feedback_score,
        regeneration=False,
        preprocessed_result=preprocessing_flow_state,
        rag_result=rag_flow_state
      )
    feedback_result = feedback_agent.invoke(feedback_flow_state)
    print("Regeneration Result: ",feedback_result['regeneration'])
    feedback_elapsed_time = time.time() - feedback_start
    print(f"Elapsed Time Including Feedback: {elapsed+feedback_elapsed_time:.2f} seconds")
  print("---------------------------")
  # count += 1
  # if count == 5:
  #   break

# print(len(list(preprocessing_store.yield_keys())))
# print(len(list(rag_store.yield_keys())))